# Обработка табличных данных

In [2]:
from typing import List

import datetime
import os
import numpy as np
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.manifold import Isomap, TSNE
# import pdb

In [7]:
def table_preprocessing(input_dataframe: str, 
                        numeric_columns: List[str] = [], 
                        categorical_columns: List[str] = [], 
                        target_columns: List[str] = [], 
                        ignore_columns: List[str] = [], 
                        unknown_column_action: str = 'infer',
                        numeric_threshold: float = 0.05,
                        numeric_scaling: str  = 'standard', 
                        categorical_encoding: str = 'one-hot',
                        nan_action: str = 'infer', 
                        nan_threshold: float = 0.5,
                        verbose: bool = True,
                        save: bool = False,
                        save_directory: str = "./"):
    """Кодирование табличных данных

    Args:
        input_dataframe (str): 
            путь до датафрейма для предобработки
        numeric_columns (List[str], optional): 
            Defaults to [].
        categorical_columns (List[str], optional): 
            Defaults to [].
        target_columns (List[str], optional): 
            Defaults to [].
        ignore_columns (List[str], optional): 
            Defaults to [].
        unknown_column_action (str, optional): 
            Options: 'infer', 'ignore'. ignore Defaults to 'infer'.
        numeric_threshold (float, optional):
            Defaults to 0.05.
        numeric_scaling (str, optional): 
            Options: 'infer', 'ignore'. Defaults to 'standard'.
        categorical_encoding (str, optional): 
            Options: 'label', 'one-hot'. Defaults to 'one-hot'.
        nan_action (str, optional): 
            Defaults to 'infer'.
        nan_threshold (float, optional): 
            Defaults to 0.5.
        verbose (bool, optional): 
            Defaults to True.
        save (bool, optional): 
            Defaults to False.
        save_directory (str, optional): 
            Defaults to "./".
        
    Returns:
        pd.DataFrame: предобработанная таблица
    """
    
    if verbose:
        print(f"--------------------------\nПредобработка\n--------------------------\n\n"
        f"\tinput_path: {input_dataframe}, save_directory: {save_directory}\n"
        f"\tnumeric_columns: {numeric_columns}, categorical_columns: {categorical_columns}, target_columns: {target_columns}, \n"
        f"\tunknown_column_action: {unknown_column_action}, ignore_columns: {ignore_columns}, \n"
        f"\tnumeric_threshold: {numeric_threshold}, numeric_scaling: {numeric_scaling}, \n"
        f"\tcategorical_encoding: {categorical_encoding}, nan_action: {nan_action}, \n"
        f"\tnan_threshold: {nan_threshold}, verbose: {verbose}, \n")

    # Дописывание пустых параметров
    if save_directory is None:
        save_directory = './'
    if os.path.exists(save_directory) is False:
        os.mkdir(save_directory)
        print(f"{datetime.datetime.now()}: Output directory created: {save_directory}.")
    
    if verbose: 
        print(f"{datetime.datetime.now()}: Output path for the preprocessed file: {save_directory}.")

    # Загрузка датафрейма
    if isinstance(input_dataframe, str):
        if input_dataframe.endswith('.csv'):
            peek_df = pd.read_csv(input_dataframe, nrows=1)
            if peek_df.columns[0].startswith('Unnamed') or peek_df.columns[0].isdigit():
                df = pd.read_csv(input_dataframe, index_col=0)
            else:
                df = pd.read_csv(input_dataframe)
        elif input_dataframe.endswith('.xlsx'):
            df = pd.read_excel(input_dataframe, index_col=None)
        elif input_dataframe.endswith('.pickle'):
            df = pd.read_pickle(input_dataframe)
        elif input_dataframe.endswith('.json'):
            df = pd.read_json(input_dataframe)
        elif input_dataframe.endswith('.parquet'):
            df = pd.read_parquet(input_dataframe)
        elif input_dataframe.endswith('.hdf') or input_dataframe.endswith('.h5'):
            df = pd.read_hdf(input_dataframe)
        else:
            supported_formats = ", ".join(["CSV", "Excel (.xlsx)", "Pickle", "JSON", "Parquet", "HDF5 (.hdf, .h5)"])
            raise ValueError(f"The file format is not supported. Please convert your file to one of the following supported formats: {supported_formats}.")
    elif isinstance(input_dataframe, pd.DataFrame):
        df = input_dataframe.copy()
    else:
        raise ValueError("Invalid input_path. Must be a path to a file or a pandas DataFrame.")

    # Checking columns
    ## Checking target_columns
    if target_columns is not None:
        if type(target_columns) != list:
            if target_columns in df.columns is False:
                raise ValueError(f"Target column {target_columns} not found.") 
            target_columns = [target_columns] # We need them to be lists. 
        else:
            for target_col in target_columns:
                if target_col in df.columns is False:
                    raise ValueError(f"Target column {target_col} not found.") 
    else:
        target_columns = []

    ## Checking numeric_columns
    if numeric_columns is not None:
        if type(numeric_columns) != list:
            if numeric_columns in df.columns is False:
                raise ValueError(f"Numeric column {numeric_columns} not found.") 
            numeric_columns = [numeric_columns] # We need them to be lists. 
        else:
            for numeric_col in numeric_columns:
                if numeric_col in df.columns is False:
                    raise ValueError(f"Numeric column {numeric_col} not found.") 
    else:
        numeric_columns = []

    ## Checking categorical_columns
    if categorical_columns is not None:
        if type(categorical_columns) != list:
            if categorical_columns in df.columns is False:
                raise ValueError(f"Categorical column {categorical_columns} not found.") 
            categorical_columns = [categorical_columns] # We need them to be lists. 
        else:
            for categorical_col in categorical_columns:
                if categorical_col in df.columns is False:
                    raise ValueError(f"Categorical column {categorical_col} not found.") 
    else:
        categorical_columns = []

    ## Checking ignore_columns
    if ignore_columns is not None:
        if type(ignore_columns) != list:
            if ignore_columns in df.columns is False:
                raise ValueError(f"Ignore column {ignore_columns} not found.") 
            ignore_columns = [ignore_columns] # We need them to be lists. 
        else:
            for ignore_col in ignore_columns:
                if ignore_col in df.columns is False:
                    raise ValueError(f"Ignore column {ignore_col} not found.") 
    else:
        ignore_columns = []

    # Targets should not be preprocessed
    ignore_columns += target_columns
    
    # Unknown columns inference
    if unknown_column_action == 'infer':
        for col in df.columns:
            if col not in numeric_columns and col not in categorical_columns and col not in ignore_columns:
                if df[col].dtype in [np.float64, np.float32, np.int64, np.int32]:
                    numeric_columns.append(col)
                    if verbose:
                        print(f"{datetime.datetime.now()}: Column '{col}' added to numeric columns by inference.")
                elif df[col].dtype == 'bool' or np.issubdtype(df[col].dtype, np.datetime64):
                    ignore_columns.append(col)
                    if verbose:
                        print(f"{datetime.datetime.now()}: Column '{col}' added to ignored columns by inference.")
                elif df[col].dtype == 'object':
                    categorical_columns.append(col)
                    if verbose:
                        print(f"{datetime.datetime.now()}: Column '{col}' added to categorical column columns by inference.")      
                else:
                    unique_ratio = len(df[col].unique()) / len(df[col])
                    if unique_ratio > numeric_threshold:
                        numeric_columns.append(col)
                        if verbose:
                            print(f"{datetime.datetime.now()}: Column '{col}' added to numeric columns by unique ratio inference.")
                    else:
                        categorical_columns.append(col)
                        if verbose:
                            print(f"{datetime.datetime.now()}: Column '{col}' added to categorical columns by unique ratio inference.")
    elif unknown_column_action == 'ignore':
        ignore_columns += [col for col in df.columns if col not in numeric_columns and col not in categorical_columns and col not in ignore_columns]
    else: 
        raise ValueError(f"unknown_column_action {unknown_column_action} not supported. Aborting...")
    if verbose:
        print(f"--------------------------\nDataframe short report\n--------------------------\n\n")
        print(f"{df.shape[0]} rows and {df.shape[1]} columns")
        print(f"column list: {list(df.columns)}")
        print(f"nans:\n{df.isna().sum()}")

    # Set target columns to be only one colum
    target_col_name = tuple(target_columns) if len(target_columns) > 1 else (target_columns[0] if len(target_columns) == 1 else '')
    if len(target_columns) > 1:
        df[target_col_name] = df[target_columns].apply(tuple, axis=1)
        df = df.drop(columns=target_columns)
    
    if len(target_columns) != 0:
        unique_targets = np.unique(df[target_col_name].values)
        N_col = df.shape[0]
        print(f"Target class proportions")
        for target in unique_targets:
            n_target = df[df[target_col_name] == target].shape[0]
            print(f"\t{target}: {n_target / N_col * 100}%")
    print(f"--------------------------\nEnd of the report.")

    # NaNs
    if nan_action == 'drop row':
        df.dropna(inplace=True)
        if verbose:
            print(f"{datetime.datetime.now()}: Dropped rows with NaN values.")
    elif nan_action == 'drop column':
        df.dropna(axis=1, thresh=int(nan_threshold * df.shape[0]), inplace=True)
        if verbose:
            print(f"{datetime.datetime.now()}: Dropped columns with NaN values above threshold.")
    elif nan_action == 'infer':
        for col in numeric_columns:
            df[col] = df[col].fillna(df[col].mean())
            if verbose:
                print(f"{datetime.datetime.now()}: Filled NaN values in numeric column '{col}' with mean.")
        for col in categorical_columns:
            df[col] = df[col].fillna(df[col].mode()[0])
            if verbose:
                print(f"{datetime.datetime.now()}: Filled NaN values in categorical column '{col}' with mode.")
        if verbose:
            print(f"{datetime.datetime.now()}: Filled NaN values with column means.")

    # Preprocessing numerical cols
    if numeric_scaling == 'standard':
        scaler = StandardScaler()
    elif numeric_scaling == 'minmax':
        scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    if verbose:
        print(f"{datetime.datetime.now()}: Scaled numeric columns using {numeric_scaling} scaling.")

    # Preprocessing cat cols
    if categorical_encoding == 'one-hot':
        df = pd.get_dummies(df, columns=categorical_columns)
    elif categorical_encoding == 'label':
        encoder = LabelEncoder()
        for col in categorical_columns:
            df[col] = encoder.fit_transform(df[col])
    if verbose:
        print(f"{datetime.datetime.now()}: Encoded categorical columns using {categorical_encoding} encoding.")

    # Сохранение
    if save:
        if save_directory.endswith('.pickle'):
            df.to_pickle(save_directory)
        elif save_directory.endswith('.csv'):
            df.to_csv(save_directory, index=False)
        elif save_directory.endswith('.xlsx'):
            df.to_excel(save_directory, index=False)
        elif save_directory.endswith('.json'):
            df.to_json(save_directory, index=False)
        elif save_directory.endswith('.parquet'):
            df.to_parquet(save_directory, index=False)
        elif save_directory.endswith('.hdf') or save_directory.endswith('.h5'):
            df.to_hdf(save_directory, index=False)
        if verbose:
            print(f"{datetime.datetime.now()}: Saved preprocessed DataFrame to {save_directory}.")

    return df

# Тестирование

In [6]:
from datasets import Dataset
import pandas as pd

# create a Pandas DataFrame
df = pd.read_csv("../data/tested.csv")

print(df.isnull().sum())
print()
print(df.dtypes)

C:\Users\Никита\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [9]:
new_df = table_preprocessing('../data/tested.csv', save_directory='../data/results', categorical_encoding='label')

--------------------------
Предобработка
--------------------------

	input_path: ../data/tested.csv, save_directory: ../data/results
	numeric_columns: [], categorical_columns: [], target_columns: [], 
	unknown_column_action: infer, ignore_columns: [], 
	numeric_threshold: 0.05, numeric_scaling: standard, 
	categorical_encoding: label, nan_action: infer, 
	nan_threshold: 0.5, verbose: True, 

2024-09-18 14:53:20.466447: Output directory created: ../data/results.
2024-09-18 14:53:20.466447: Output path for the preprocessed file: ../data/results.
2024-09-18 14:53:20.470195: Column 'PassengerId' added to numeric columns by inference.
2024-09-18 14:53:20.470195: Column 'Survived' added to numeric columns by inference.
2024-09-18 14:53:20.470195: Column 'Pclass' added to numeric columns by inference.
2024-09-18 14:53:20.471660: Column 'Name' added to categorical column columns by inference.
2024-09-18 14:53:20.471660: Column 'Sex' added to categorical column columns by inference.
2024-09-18

In [11]:
print(new_df.isnull().sum())
print()
print(new_df.dtypes)

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

PassengerId    float64
Survived       float64
Pclass         float64
Name             int32
Sex              int32
Age            float64
SibSp          float64
Parch          float64
Ticket           int32
Fare           float64
Cabin            int32
Embarked         int32
dtype: object


In [12]:
new_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,-1.727912,-0.755929,0.873482,206,1,0.334993,-0.499470,-0.400248,152,-0.498407,15,1
1,-1.719625,1.322876,0.873482,403,0,1.325530,0.616992,-0.400248,221,-0.513274,15,2
2,-1.711337,-0.755929,-0.315819,269,1,2.514175,-0.499470,-0.400248,73,-0.465088,15,1
3,-1.703050,-0.755929,0.873482,408,1,-0.259330,-0.499470,-0.400248,147,-0.483466,15,2
4,-1.694763,1.322876,0.873482,178,0,-0.655545,0.616992,0.619896,138,-0.418471,15,2
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1.694763,-0.755929,0.873482,353,1,0.000000,-0.499470,-0.400248,267,-0.494448,15,2
414,1.703050,1.322876,-1.505120,283,0,0.691586,-0.499470,-0.400248,324,1.313753,22,0
415,1.711337,-0.755929,0.873482,332,1,0.651965,-0.499470,-0.400248,346,-0.508792,15,2
416,1.719625,-0.755929,0.873482,384,1,0.000000,-0.499470,-0.400248,220,-0.494448,15,2
